In [1]:
import cv2
import numpy as np
import os
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
import csv


Load Data (F & L)

In [3]:

# Load features for model training
def load_features(file_path):
    data = []
    labels = []

    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            features = list(map(float, row[1:-1]))  # Exclude file name and label
            label = 0 if "control" in row[0].lower() else 1  # Infer label from file name
            data.append(features)
            labels.append(label)

    return np.array(data), np.array(labels)

output_csv_path = "pd_features.csv"

features, labels = load_features(output_csv_path)

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)




FileNotFoundError: [Errno 2] No such file or directory: 'pd_features.csv'

In [36]:
# Preprocess features
scaler = StandardScaler()
features = scaler.fit_transform(features)

print(scaler)
print(features)


StandardScaler()
[[ 1.15844938  1.16824525  1.16457344  0.96544221  0.95679163  1.04682006
   0.8560265   1.0440808   0.61159908]
 [-0.31143177 -0.30819178 -0.31050064 -0.11653894 -0.11479099 -0.11984925
   0.04007495  0.03221609  0.00683206]
 [-0.34658811 -0.38054186 -0.3173827  -0.29616704 -0.27328537 -0.31730087
  -0.4106434  -0.35100434 -0.41273042]
 [-0.44490609 -0.46260025 -0.41900972 -0.49055247 -0.47198142 -0.52479953
  -0.60035119 -0.58465453 -0.6037477 ]
 [-0.25760195 -0.26869006 -0.21435435 -0.46673447 -0.43232942 -0.51427085
  -0.66954393 -0.61398086 -0.67128743]
 [-0.78155846 -0.76444119 -0.84372301 -0.40803822 -0.39580633 -0.43731974
  -0.42331944 -0.46058092 -0.36966991]
 [-0.71769601 -0.7268095  -0.7337055  -0.4828769  -0.46468838 -0.49841761
  -0.62842613 -0.63055934 -0.59770511]
 [-0.39483148 -0.41020247 -0.36766174 -0.5271088  -0.49551541 -0.56058335
  -0.70292322 -0.64738581 -0.70389557]
 [ 0.06605052  0.01593011  0.16628963  0.10039501 -0.02829809  0.27876124
   0.

In [37]:
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)


Features shape: (25, 9)
Labels shape: (25,)


In [38]:
from sklearn.model_selection import StratifiedShuffleSplit

# Stratified splitting to maintain class balance
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state= 42)
for train_idx, test_idx in splitter.split(features, labels):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Verify class distribution
print("Training class distribution:", np.bincount(y_train))
print("Testing class distribution:", np.bincount(y_test))

# from sklearn.model_selection import StratifiedShuffleSplit

# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=7)
# for train_idx, test_idx in splitter.split(features, labels):
#     X_train, X_test = features[train_idx], features[test_idx]
#     y_train, y_test = labels[train_idx], labels[test_idx]

# print("Training class distribution:", np.bincount(y_train))
# print("Testing class distribution:", np.bincount(y_test))




X_train shape: (17, 9)
X_test shape: (8, 9)
y_train shape: (17,)
y_test shape: (8,)
Training class distribution: [ 0 17]
Testing class distribution: [0 8]


In [39]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [40]:
# print("Training class distribution:", np.bincount(y_train))
# print("Testing class distribution:", np.bincount(y_test))


In [41]:
# print("Sample X_train:", X_train[:5])
# print("Sample y_train:", y_train[:5])


In [43]:
import matplotlib.pyplot as plot
plot(f=data[35],  plot_func=sns.barplot, t_id=0, x='Timestamp', y='Pressure')

NameError: name 'data' is not defined

Split DaTA (Test and Train)

In [ ]:
# # Train-test split
# np.random.seed(42)
# pos_indices = np.where(labels == 1)[0]
# neg_indices = np.where(labels == 0)[0]

# np.random.shuffle(pos_indices)
# np.random.shuffle(neg_indices)

# train_pos = pos_indices[:-5]
# test_pos = pos_indices[-5:]
# train_neg = neg_indices[:-5]
# test_neg = neg_indices[-5:]

# train_indices = np.concatenate([train_pos, train_neg])
# test_indices = np.concatenate([test_pos, test_neg])

# X_train = features[train_indices]
# y_train = labels[train_indices]
# X_test = features[test_indices]
# y_test = labels[test_indices]

# print("Training set shape:", X_train.shape)
# print("Testing set shape:", X_test.shape)



In [ ]:
# Metrics function
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%\n")

### Classification ###

In [ ]:
# # Splitting the dataset into positive and negative cases
# pos_indices = np.where(labels == 1)[0]
# neg_indices = np.where(labels == 0)[0]

# # Shuffle indices to ensure randomness
# np.random.shuffle(pos_indices)
# np.random.shuffle(neg_indices)

# # Split the data into training and testing sets
# train_pos = pos_indices[:-5]
# test_pos = pos_indices[-5:]
# train_neg = neg_indices[:-5]
# test_neg = neg_indices[-5:]

# # Create training and testing datasets
# train_indices = np.concatenate([train_pos, train_neg])
# test_indices = np.concatenate([test_pos, test_neg])

# X_train = features[train_indices]
# y_train = labels[train_indices]
# X_test = features[test_indices]
# y_test = labels[test_indices]

# print("Training set shape:", X_train.shape)
# print("Testing set shape:", X_test.shape)

In [ ]:
def accuracy(prediction,actual):
    correct = 0
    not_correct = 0
    for i in range(len(prediction)):
        if prediction[i] == actual[i]:
            correct+=1
        else:
            not_correct+=1
    return (correct*100)/(correct+not_correct)


def metrics(prediction,actual):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(prediction)):
        if prediction[i] == actual[i] and actual[i]==1:
            tp+=1
        if prediction[i] == actual[i] and actual[i]==0:
            tn+=1
        if prediction[i] != actual[i] and actual[i]==0:
            fp+=1
        if prediction[i] != actual[i] and actual[i]==1:
            fn+=1
    metrics = {'Precision':(tp/(tp+fp+tn+fn)),'Recall':(tp/(tp+fn)),'F1':(2*(tp/(tp+fp+tn+fn))*(tp/(tp+fn)))/((tp/(tp+fp+tn+fn))+(tp/(tp+fn)))}
    return (metrics)

Logistic regression

In [ ]:
# Train Logistic Regression
def train_logistic_regression(X_train, y_train, X_test, y_test):
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    false_positive_rate = calculate_false_positive_rate(y_test, y_pred)
    print(f"Logistic Regression Accuracy: {accuracy * 100:.2f}%")
    print(f"Logistic Regression False Positive Rate: {false_positive_rate * 100:.2f}%\n")






In [ ]:
# Train Random Forest
def train_random_forest(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    false_positive_rate = calculate_false_positive_rate(y_test, y_pred)
    print(f"Random Forest Accuracy: {accuracy * 100:.2f}%")
    print(f"Random Forest False Positive Rate: {false_positive_rate * 100:.2f}%\n")

In [ ]:
# Train K-Nearest Neighbors
def train_knn(X_train, y_train, X_test, y_test):
    clf = KNeighborsClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    false_positive_rate = calculate_false_positive_rate(y_test, y_pred)
    print(f"KNN Accuracy: {accuracy * 100:.2f}%")
    print(f"KNN False Positive Rate: {false_positive_rate * 100:.2f}%\n")

In [ ]:
# Train Support Vector Machine
def train_svm(X_train, y_train, X_test, y_test):
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    false_positive_rate = calculate_false_positive_rate(y_test, y_pred)
    print(f"SVM Accuracy: {accuracy * 100:.2f}%")
    print(f"SVM False Positive Rate: {false_positive_rate * 100:.2f}%\n")

In [ ]:
# Train XGBoost
def train_xgboost(X_train, y_train, X_test, y_test):
    clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    false_positive_rate = calculate_false_positive_rate(y_test, y_pred)
    print(f"XGBoost Accuracy: {accuracy * 100:.2f}%")
    print(f"XGBoost False Positive Rate: {false_positive_rate * 100:.2f}%\n")

In [ ]:
# Train and evaluate models separately
train_logistic_regression(X_train, y_train, X_test, y_test)
train_random_forest(X_train, y_train, X_test, y_test)
train_knn(X_train, y_train, X_test, y_test)
train_svm(X_train, y_train, X_test, y_test)
train_xgboost(X_train, y_train, X_test, y_test)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)

In [ ]:

# # Classifier training and evaluation
# classifiers = {
#     "Logistic Regression": LogisticRegression(),
#     "Random Forest": RandomForestClassifier(),
#     "Support Vector Machine": SVC(),
#     "K-Nearest Neighbors": KNeighborsClassifier(),
#     "XGBoost": XGBClassifier(),
#     "Decision Tree": DecisionTreeClassifier(),
# }

# for name, clf in classifiers.items():
#     print(f"Training {name}...")
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     evaluate_model(y_test, y_pred, name)

